<a href="https://colab.research.google.com/github/surensnyper/class_machine_vision/blob/main/RF-Multiclassification2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [1]:
!pip install transformers

     |████████████████████████████████| 4.0 MB 4.2 MB/s 
     |████████████████████████████████| 895 kB 72.8 MB/s 
     |████████████████████████████████| 77 kB 8.1 MB/s 
     |████████████████████████████████| 596 kB 70.3 MB/s 
     |████████████████████████████████| 6.6 MB 73.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [56]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [57]:
# df = pd.read_csv(
#     '/content/smile-annotations-final.csv', 
#     names=['id',	'text', 'category'])
# df.set_index('id', inplace=True)

df = pd.read_excel(
    '/content/drive/MyDrive/Colab Notebooks/Classification/Multiclassificaion/Coursera_Freq_Total_Clean_EDA.xlsx', 
    names=['Index',	'Paper', 'Abstract', 'Class'])
df.set_index('Index', inplace=True)

In [58]:
df.head()

,Paper,Abstract,Class
Index,,,
2,Radiofrequency Electromagnetic Field Have No E...,The intracranial 9L tumor model was used to de...,800MHz_to_1GHz
3,Chronic Exposure of Cancer-Prone Mice to Low-L...,The purpose of this study was to determine whe...,1_to_3GHz
4,Long Term Low Level Exposure of Mice Prone to ...,The purpose of this study was to determine if ...,Below_800MHz
5,Long Term Low Level Exposure of Mice Prone to ...,the intent of this study was to determine if c...,Below_800MHz
6,Long Term Low Level Exposure of Mice Prone to ...,the intent this study was to determine if cont...,Below_800MHz


In [59]:
df.Abstract.iloc[0]

'The intracranial 9L tumor model was used to determine if exposure to a radiofrequency (RF) electromagnetic field similar to those used in cellular telephone has any effects on the growth of a central nervous system tumor. Fischer 344 rats implanted with different numbers of 9L gliosarcoma cells were exposed to 835.62 MHz frequency-modulated continuous wave (FMCW) or 847.74 MHz code division multiple access (CDMA) RF field with nominal slot-average specific absorption rates in the brain of 0.75 +/- 0.25 W/kg. The animals were exposed to the RF field for 4 h a day, 5 days a week starting 4 weeks prior to and up to 150 days after the implantation of tumor cells. Among sham-exposed animals injected with 2 to 10 viable cells (group 1), the median survival was 70 days, with 27% of the animals surviving at 150 days. The median survival length and final survival fraction for animals injected with 11 to 36 viable cells (group 2) were 52 days and 14%, respectively, while the values for those in

In [60]:
df.Class.value_counts()

800MHz_to_1GHz    155
1_to_3GHz         154
Below_800MHz       48
Above_6GHz         42
3_to_6GHz          32
Name: Class, dtype: int64

In [61]:
possible_labels = df.Class.unique()

In [62]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
  label_dict[possible_label] = index

In [63]:
label_dict

{'1_to_3GHz': 1,
 '3_to_6GHz': 4,
 '800MHz_to_1GHz': 0,
 'Above_6GHz': 3,
 'Below_800MHz': 2}

In [64]:
df['label'] = df.Class.replace(label_dict)
# df.head()
df.head(10)

,Paper,Abstract,Class,label
Index,,,,
2,Radiofrequency Electromagnetic Field Have No E...,The intracranial 9L tumor model was used to de...,800MHz_to_1GHz,0
3,Chronic Exposure of Cancer-Prone Mice to Low-L...,The purpose of this study was to determine whe...,1_to_3GHz,1
4,Long Term Low Level Exposure of Mice Prone to ...,The purpose of this study was to determine if ...,Below_800MHz,2
5,Long Term Low Level Exposure of Mice Prone to ...,the intent of this study was to determine if c...,Below_800MHz,2
6,Long Term Low Level Exposure of Mice Prone to ...,the intent this study was to determine if cont...,Below_800MHz,2
7,Long Term Low Level Exposure of Mice Prone to ...,the intent of this study was to determine if c...,Below_800MHz,2
8,Long Term Low Level Microwave Irradiation of Rats,Our goal was to investigate effects of long-te...,1_to_3GHz,1
9,Acceleration of Development of Benzopyrene-Ind...,Development and growth of skin cancer may be a...,1_to_3GHz,1
10,Accelerated Deveopment of Spotaneousand Benzop...,Development and growth of skin cancer may be a...,1_to_3GHz,1


Data imbalance consideration

In [102]:
from sklearn.model_selection import train_test_split

In [103]:
X_train, X_val, y_train, y_val = train_test_split(
    df.index.values,
    df.label.values,
    test_size = 0.3,
    random_state = 17,
    stratify = df.label.values
)

In [104]:
df['data_type'] = ['not_set']*df.shape[0]

In [105]:
df.head()

,Paper,Abstract,Class,label,data_type
Index,,,,,
2,Radiofrequency Electromagnetic Field Have No E...,The intracranial 9L tumor model was used to de...,800MHz_to_1GHz,0,not_set
3,Chronic Exposure of Cancer-Prone Mice to Low-L...,The purpose of this study was to determine whe...,1_to_3GHz,1,not_set
4,Long Term Low Level Exposure of Mice Prone to ...,The purpose of this study was to determine if ...,Below_800MHz,2,not_set
5,Long Term Low Level Exposure of Mice Prone to ...,the intent of this study was to determine if c...,Below_800MHz,2,not_set
6,Long Term Low Level Exposure of Mice Prone to ...,the intent this study was to determine if cont...,Below_800MHz,2,not_set


In [106]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [107]:
df.groupby(['Class', 'label', 'data_type']).count()

Paper  Abstract
Class          label data_type                 
1_to_3GHz      1     train        108       108
                     val           46        46
3_to_6GHz      4     train         22        22
                     val           10        10
800MHz_to_1GHz 0     train        108       108
                     val           47        47
Above_6GHz     3     train         29        29
                     val           13        13
Below_800MHz   2     train         34        34
                     val           14        14

In [108]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [109]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case = True
)

In [110]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type == 'train'].Abstract.values,
    add_special_tokens = True,
    return_attention_mask = True,
    pad_to_max_length = True,
    max_length = 256,
    return_tensors = 'pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type == 'val'].Abstract.values,
    add_special_tokens = True,
    return_attention_mask = True,
    pad_to_max_length = True,
    max_length = 256,
    return_tensors = 'pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type == 'train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type == 'val'].label.values)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [111]:
dataset_train = TensorDataset(input_ids_train,
                              attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val,
                              attention_masks_val, labels_val)

In [112]:
len(dataset_train)

301

In [113]:
len(dataset_val)

130

In [114]:
from transformers import BertForSequenceClassification

In [115]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = len(label_dict),
    output_attentions = False,
    output_hidden_states = False
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [116]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [117]:
batch_size = 4 #32 # if memory limited can be changed by 4 

dataloader_train = DataLoader(
    dataset_train,
    sampler = RandomSampler(dataset_train),
    batch_size = batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler = RandomSampler(dataset_val),
    batch_size = 32
)

In [118]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [119]:
optimizer = AdamW(
    model.parameters(),
    lr = 1e-5, # Original paper recommends between 2e-5 ~ 5e-5
    eps = 1e-8
)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [120]:
epochs = 10

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0,
    num_training_steps = len(dataloader_train)*epochs
)

In [121]:
# Prediction structure (Probabilities form to discrete)
# preds = [0.9, 0.05, 0.5, 0.03, 0.01 ]
# preds = [1, 0, 0, 0, 0] ->> We want to change this form

In [122]:
import numpy as np

In [123]:
from sklearn.metrics import f1_score

In [124]:
def f1_score_func(preds, labels):
  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()
  return f1_score(labels_flat, preds_flat, average='weighted')

In [125]:
def accuracy_per_class(preds, labels):
  label_dict_inverse = {v: k for k, v in label_dict.items()}

  preds_flat = np.argmax(preds, axis=1).flatten()
  labels_flat = labels.flatten()

  for label in np.unique(labels_flat):
    y_preds = preds_flat[labels_flat == label]
    y_true = labels_flat[labels_flat == label]
    print(f'Class: {label_dict_inverse[label]}')
    print(f'Accuracy: {len(y_preds[y_preds == label])}/{len(y_true)}\n')

In [126]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [127]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cuda


In [128]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals


In [129]:
# Before running, Make a folder 'Model' in root

for epoch in tqdm(range(1, epochs+1)):
  model.train()
  loss_train_total = 0
  progress_bar = tqdm(dataloader_train,
                      desc = 'Epoch {:1d}'.format(epoch),
                      leave = False,
                      disable = False)
  for batch in progress_bar:
    model.zero_grad()
    batch = tuple(b.to(device) for b in batch)
    inputs = {
        'input_ids'       : batch[0],
        'attention_mask'  : batch[1],
        'labels'          : batch[2]
    }
    outputs = model(**inputs)

    loss = outputs[0]
    loss_train_total += loss.item()
    loss.backward()

    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

    optimizer.step()
    scheduler.step()

    progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
  
  torch.save(model.state_dict(), f'/content/Model/BERT_finetuned_epoch{epoch}.model') # Folder build in advance

  tqdm.write('\nEpoch {epoch}')

  loss_train_avg = loss_train_total / len(dataloader_train)
  tqdm.write(f'Training loss: {loss_train_avg}')

  val_loss, predictions, true_vals = evaluate(dataloader_val)
  val_f1 = f1_score_func(predictions, true_vals)
  tqdm.write(f'Validation loss: {val_loss}')
  tqdm.write(f'F1 Score (weighted): {val_f1}')

  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/76 [00:00<?, ?it/s]


Epoch {epoch}
Training loss: 1.4183480802335238
Validation loss: 1.089867615699768
F1 Score (weighted): 0.4972109014379476


Epoch 2:   0%|          | 0/76 [00:00<?, ?it/s]


Epoch {epoch}
Training loss: 1.0372519712699086
Validation loss: 1.1204411029815673
F1 Score (weighted): 0.4976977121239417


...


Epoch {epoch}
Training loss: 0.3317817741710889
Validation loss: 0.8387429773807525
F1 Score (weighted): 0.7489045519588506


Epoch 9:   0%|          | 0/76 [00:00<?, ?it/s]


Epoch {epoch}
Training loss: 0.2987442602844615
Validation loss: 0.8077106237411499
F1 Score (weighted): 0.7479060381048135


Epoch 10:   0%|          | 0/76 [00:00<?, ?it/s]


Epoch {epoch}
Training loss: 0.2759365652148661
Validation loss: 0.7298578172922134
F1 Score (weighted): 0.7878737135879993


In [130]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = len(label_dict),
    output_attentions = False,
    output_hidden_states = False
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [131]:
model.to(device)
pass

In [132]:
model.load_state_dict(
    torch.load('/content/Model/BERT_finetuned_epoch10.model',
        # '/content/Model/finetuned_bert_epoch_1_gpu_trained.model',
               map_location = torch.device('cpu')))

<All keys matched successfully>

In [133]:
_, predictions, true_vals = evaluate(dataloader_val)

In [134]:
accuracy_per_class(predictions, true_vals)

Class: 800MHz_to_1GHz
Accuracy: 34/47

Class: 1_to_3GHz
Accuracy: 37/46

Class: Below_800MHz
Accuracy: 11/14

Class: Above_6GHz
Accuracy: 12/13

Class: 3_to_6GHz
Accuracy: 8/10



In [135]:
# Folder delete
import shutil

shutil.rmtree('/content/Model')